In [1]:
import sys
sys.path.append('3W')
sys.path.append('BibMon')


In [2]:
import os
import copy
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import missingno

In [3]:
tags = {'P-PDG':'Pa', 
        'P-TPT':'Pa', 'T-TPT':'degC',
         'P-MON-CKP':'Pa', 'T-JUS-CKP':'degC', 
         'P-JUS-CKGL':'Pa', 'T-JUS-CKGL':'degC',
         'QGL': 'sm^3/s'}

names = ['Pressure at the PDG',
         'Pressure at the TPT',
         'Temperature at the TPT',
         'Pressure upstream of the PCK',
         'Temperature downstream of the PCK',
         'Pressure downstream of the GLCK',
         'Temperature downstream of the GLCK',
         'Gas lift flow rate']

reference_table = pd.DataFrame(index=np.arange(1,len(names)+1))
reference_table.index.name='Number'
reference_table['Tag'] = tags.keys()
reference_table['Name'] = names
reference_table['Unit'] = tags.values()
reference_table

Tag                                Name    Unit
Number                                                        
1            P-PDG                 Pressure at the PDG      Pa
2            P-TPT                 Pressure at the TPT      Pa
3            T-TPT              Temperature at the TPT    degC
4        P-MON-CKP        Pressure upstream of the PCK      Pa
5        T-JUS-CKP   Temperature downstream of the PCK    degC
6       P-JUS-CKGL     Pressure downstream of the GLCK      Pa
7       T-JUS-CKGL  Temperature downstream of the GLCK    degC
8              QGL                  Gas lift flow rate  sm^3/s

In [4]:
plt.rc('font', family='serif', size=12) # controls default text sizes
plt.rc('axes', titlesize=14)     # fontsize of the axes title
plt.rc('axes', labelsize=13)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=12)    # legend fontsize
plt.rc('figure', titlesize=14)  # fontsize of the figure title

In [6]:
import bibmon
import pandas as pd
import matplotlib.pyplot as plt

# df_raw = bibmon.load_3W_data(0)

import bibmon
import pandas as pd
import matplotlib.pyplot as plt

# df_raw = bibmon.load_3W_data(0)

EVENT = 0

PATH = os.path.join('..', '..', '..', '3W', 'dataset', str(EVENT))

# Inicializar a lista de arquivos
files = []

if os.path.exists(PATH):
    files = [f for f in os.listdir(PATH) if os.path.isfile(os.path.join(PATH, f))]
else:
    print(f"O caminho {PATH} não existe.")

dfs = {}

if files:  # Verifica se a lista de arquivos não está vazia
    for file in files:
        file_path = os.path.join(PATH, file)
        try:
            if file.endswith('.csv'):
                # Tentar ler o arquivo CSV com a codificação padrão
                dfs[file[:-4]] = pd.read_csv(file_path, index_col=0, parse_dates=True).rename_axis(None)
            elif file.endswith('.parquet'):
                # Ler o arquivo Parquet
                dfs[file[:-8]] = pd.read_parquet(file_path)  # Remover o sufixo '.parquet'
            else:
                print(f"Formato de arquivo desconhecido para {file}.")
        except Exception as e:
            print(f"Erro ao ler {file}: {e}")

    print('Data read!\nNumber of instances: ', len(dfs))
else:
    print("Nenhum arquivo encontrado.")



AttributeError: module 'bibmon' has no attribute 'load_3W_data'

In [30]:
%%time

i=0

for key in sorted(dfs.keys(), key=lambda x:x.lower()):
    well = int(key[8:10])
    df_test = copy.deepcopy(dfs[key])
    if 'class' in df_test.columns:
        del df_test['class']
    df_test.columns = np.arange(1,9)
    df_test.loc[:,(df_test.var(ddof=1) < 1e-6)] = np.nan
    missingno.matrix(df_test,sparkline=False,ax=ax.ravel()[i])
    ax.ravel()[i].set_yticks([])
    ax.ravel()[i].xaxis.set_tick_params(labelsize=11,rotation=0)
    ax.ravel()[i].set_title(f'{well}\n {df_test.index[0]}\n{df_test.index[-1]}',y=-0.4,fontdict={'fontsize': 12})
    i+=1
    del df_test


ValueError: Length mismatch: Expected axis has 28 elements, new values have 8 elements